In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re
from splinter import Browser
import pandas as pd
import itertools

In [ ]:
#Define Executable Path for ChromeDriver to use for web scraping

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Define base endpoint URL. Other url's will be built off this endpoint.

baseball_reference_main = 'https://www.baseball-reference.com'
nats_roster_endpoint = '/teams/WSN/2021-roster.shtml'
nats_roster_page = baseball_reference_main + nats_roster_endpoint
browser.visit(nats_roster_page)

In [ ]:
#Use Beautiful Soup HTML.parser to extract relvant details about news articles

roster_html = browser.html
roster_soup = bs(roster_html, 'html.parser')
roster_soup

In [ ]:
testing = roster_soup.find("a", string="Trea Turner")
testing_url_string = testing['href']
overall_string = baseball_reference_main + testing_url_string
overall_string

In [ ]:
all_nats_players_this_season_html = roster_soup.find('table', class_="sortable stats_table shade_zero now_sortable", id="appearances").find('tbody').find_all(attrs={"data-stat": "player"})

In [ ]:
active_40_man_df = pd.read_html(nats_roster_page,keep_default_na=False)[0]
active_40_man_df = active_40_man_df[active_40_man_df.Name != 'Name']
active_40_man_df

In [ ]:
active_40_man_df['NatsOrgFlag'] = 1

In [ ]:
active_40_man_df = active_40_man_df.replace('*',1)
active_40_man_df

In [ ]:
active_40_man_col_rename = active_40_man_df.rename(columns={"Unnamed: 3": "Country_ISO_Code","Unnamed: 4": "GenPos"},inplace = False)

In [ ]:
active_40_man_col_rename

In [ ]:
revised_country_values = active_40_man_col_rename["Country_ISO_Code"].str.slice(start=-2).to_list()
active_40_man_col_rename["Country_ISO_Code"] = revised_country_values

In [ ]:
active_40_man_col_rename = active_40_man_col_rename.drop(['Rk'],axis=1)
active_40_man_col_rename

In [ ]:
non_40_man_df = pd.read_html(nats_roster_page,keep_default_na=False)[1]
non_40_man_df

In [ ]:
all_season_roster = active_40_man_col_rename.merge(non_40_man_df, on='Name', how='outer')
all_season_roster

In [ ]:
all_season_roster_list = all_season_roster['Name'].to_list()
all_season_roster_list

In [ ]:
all_season_roster.columns

In [ ]:
all_season_filtered_columns = all_season_roster[['Name','OnActv','IL','Age_y','Unnamed: 2', 'B_y', 'T_y',
                                       'Ht_y','Wt_y','DoB_y','G','GS','P','C','1B', '2B', '3B', 
                                        'SS', 'LF', 'CF','RF', 'OF', 'DH', 'PH', 'PR','NatsOrgFlag']]

In [ ]:
all_season_newColnames = all_season_filtered_columns.rename({'Age_y': 'Age', 'Unnamed: 2': 'Country_Code', 'B_y':'B',
                           'T_y':'T','Ht_y':'Ht','Wt_y':'Wt','DoB_y':'DoB'}, axis=1)
all_season_newColnames

In [ ]:
#Some cleaning activities
revised_country_values = all_season_newColnames["Country_Code"].str.slice(start=-2).to_list()
all_season_newColnames["Country_Code"] = revised_country_values

In [ ]:
all_season_newColnames['OnActv'] = all_season_newColnames['OnActv'].fillna(0).to_list()
all_season_newColnames['IL'] = all_season_newColnames['IL'].fillna('').to_list()

all_season_newColnames

In [ ]:
non_forty_guys_df = all_season_newColnames.loc[(all_season_newColnames.NatsOrgFlag != 1)] 
non_forty_guys_df

In [ ]:
non_forty_guys_names = non_forty_guys_df['Name'].to_list()
non_forty_guys_names

In [ ]:
try:
    non_forty_guys_names.remove('Name')
except ValueError:
    pass

non_forty_guys_names

In [ ]:
team_list_non40_names = []
team_list_non40_teams = []
all_player_images = []
limit = 1
for index, player in enumerate(all_season_roster_list):
    if index == limit:
        break
    try:
        player_source_tag = roster_soup.find("a", string='Stephen Strasburg')
        player_url_endpoint = player_source_tag['href']
        player_profile = baseball_reference_main + player_url_endpoint
        print(player)
        browser.visit(player_profile)
        profile_html = browser.html
        profile_soup = bs(profile_html, 'html.parser')
        team_string_header = profile_soup.find('div',class_='players').find(attrs={"itemtype":"https://schema.org/Person"}).find(string='Team:').find_parent('p')
        print(team_string_header)
        player_team_main = team_string_header.a.text
        print(player_team_main)
        player_team_level_parenthesis = team_string_header.contents[-1]
        start_grouping = player_team_level_parenthesis.find("(") + len("(")
        end_grouping = player_team_level_parenthesis.find(")")
        org_injury_info = player_team_level_parenthesis[start_grouping:end_grouping]
        org_injury_list = org_injury_info.split(",")
        print(org_injury_list)
        
        if "majors" in org_injury_list:
            print('Should have a 25 man flag & 40 man flag')
        elif "40-man" in org_injury_list:
            print('Should have a 40 man flag ONLY')
        else:
            print('Should not have a 25 man flag but could be 40 man')
            
        if "IL" in org_injury_list[-1]:
            print('Player is injured')
            print(org_injury_list[-1])
        player_image_tag = profile_soup.find('div', id = 'info').find('div',class_ = 'media-item multiple').img['src']
        print(player_image_tag)
        
#         if player in non_forty_guys_names:
#             player_source_tag = roster_soup.find("a", string=player)
#             player_url_endpoint = player_source_tag['href']
#             player_profile = baseball_reference_main + player_url_endpoint
#             print(player)
#             browser.visit(player_profile)
#             profile_html = browser.html
#             profile_soup = bs(profile_html, 'html.parser')
#             player_image_tag = profile_soup.find('div', id = 'info').find('div',class_ = 'media-item multiple').img['src']
#             print(player_image_tag)
#             all_player_images.append(player_image_tag)
#             try:
#                 team_string_header = profile_soup.find('div',class_='players').find(attrs={"itemtype":"https://schema.org/Person"}).find(string='Team:')
#                 team_parent = team_string_header.find_parent('p').a.text
#                 print(team_parent) 
#                 if team_parent == 'Washington Nationals':
#                     team_list_non40_names.append(player)
#                 else:
#                     pass
#             except AttributeError:
#                 print('Free Agent or Released')
#         elif player in all_season_roster_list:
#             player_source_tag = roster_soup.find("a", string=player)
#             player_url_endpoint = player_source_tag['href']
#             player_profile = baseball_reference_main + player_url_endpoint
#             print(player)
#             browser.visit(player_profile)
#             profile_html = browser.html
#             profile_soup = bs(profile_html, 'html.parser')
#             player_image_tag = profile_soup.find('div', id = 'info').find('div',class_ = 'media-item multiple').img['src']
#             all_player_images.append(player_image_tag)
#             print(player_image_tag)
#             print('Washington Nationals')
#         else:
#             all_player_images.append('Placeholder. Possible Error')
    except AttributeError:
        print('AttributeError')
        all_player_images.append('Placeholder. Possible Error')

In [ ]:
print(team_list_non40_names)

In [ ]:
print(all_player_images)

In [ ]:
for name in team_list_non40_names:
    all_season_newColnames.loc[all_season_newColnames.Name == name, 'NatsOrgFlag'] = 1
    
all_season_newColnames

In [ ]:
len(all_player_images)

In [ ]:
all_season_newColnames['Img_URL'] = all_player_images
all_season_newColnames

In [ ]:
all_season_players_org_divided = all_season_newColnames[all_season_newColnames.Name != 'Name']

In [ ]:
forty_man_never_mlb = all_season_players_org_divided.loc[all_season_players_org_divided.Age.isna() == True]
forty_man_never_mlb

In [ ]:
forty_man_never_mlb_list = forty_man_never_mlb['Name'].to_list()
forty_man_never_mlb_list

In [ ]:
for player_name in forty_man_never_mlb_list:
    print(player_name)
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'Country_Code'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'Country_ISO_Code'].iloc[0]
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'B'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'B_x'].iloc[0]
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'T'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'T_x'].iloc[0]
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'Ht'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'Ht_x'].iloc[0]
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'Wt'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'Wt_x'].iloc[0]
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'DoB'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'DoB_x'].iloc[0]
    all_season_players_org_divided.loc[(all_season_players_org_divided.Name == player_name),'Age'] = all_season_roster.loc[all_season_roster['Name'] == player_name,'Age_x'].iloc[0]
    

In [ ]:
#Confirmation of correct mapping
all_season_players_org_divided.loc[all_season_players_org_divided.Name == 'Steven Fuentes']

In [ ]:
all_season_players_org_divided['G'] = all_season_players_org_divided['G'].fillna(0).to_list()
all_season_players_org_divided['GS'] = all_season_players_org_divided['GS'].fillna(0).to_list()
all_season_players_org_divided['P'] = all_season_players_org_divided['P'].fillna(0).to_list()
all_season_players_org_divided['1B'] = all_season_players_org_divided['1B'].fillna(0).to_list()
all_season_players_org_divided['2B'] = all_season_players_org_divided['2B'].fillna(0).to_list()
all_season_players_org_divided['3B'] = all_season_players_org_divided['3B'].fillna(0).to_list()
all_season_players_org_divided['SS'] = all_season_players_org_divided['SS'].fillna(0).to_list()
all_season_players_org_divided['C'] = all_season_players_org_divided['C'].fillna(0).to_list()
all_season_players_org_divided['LF'] = all_season_players_org_divided['LF'].fillna(0).to_list()
all_season_players_org_divided['CF'] = all_season_players_org_divided['CF'].fillna(0).to_list()
all_season_players_org_divided['RF'] = all_season_players_org_divided['RF'].fillna(0).to_list()
all_season_players_org_divided['OF'] = all_season_players_org_divided['OF'].fillna(0).to_list()
all_season_players_org_divided['DH'] = all_season_players_org_divided['DH'].fillna(0).to_list()
all_season_players_org_divided['PH'] = all_season_players_org_divided['PH'].fillna(0).to_list()
all_season_players_org_divided['PR'] = all_season_players_org_divided['PR'].fillna(0).to_list()
all_season_players_org_divided['NatsOrgFlag'] = all_season_players_org_divided['NatsOrgFlag'].fillna(0).to_list()

In [ ]:
all_season_players_org_divided.loc[all_season_players_org_divided['Name'] == 'Steven Fuentes']

In [ ]:
all_season_players_org_divided.isnull().values.any()

In [ ]:
all_season_players_org_divided['NatsOrgFlag'] = all_season_players_org_divided['NatsOrgFlag'].astype(int).to_list()
all_season_players_org_divided

In [ ]:
try:
    all_season_players_org_divided.insert(0, 'Player_ID', range(0, 0 + len(all_season_players_org_divided)))
except ValueError:
    pass

all_season_players_org_divided

In [ ]:
all_season_players_org_divided.to_csv('season_long_roster', index=False) 